In [1]:
import pandas as pd
import numpy as np
import warnings
import re
from collections import OrderedDict
warnings.filterwarnings('default')

In [2]:
# подгрузка всех файлов
import os
dr = 'exp'
file_names = os.listdir(dr)

In [3]:
file_names.remove('prepared')

In [4]:
# файлы в список
df_dicts = []
for name in file_names:
    df_dicts.append([name, pd.read_excel(dr+'\\'+name, None)])
    print(name)

1квартал2017.xls
2012 1 квартал (январь, февраль, март)испр.xls
2012 2 квартал (апрель, май, июнь)испр..xls
2012 3 квартал (июль, август, сентябрь)испр..xls
2012 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2013 1 квартал (январь, февраль, март)испр.xls
2013 2 квартал (апрель, май, июнь)испр..xls
2013 3 квартал (июль, август, сентябрь)испр..xls
2013 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2014 1 квартал (январь, февраль, март)испр.xls
2014 2 квартал (апрель, май, июнь)испр.xls
2014 3 квартал (июль, август, сентябрь)испр.xls
2014 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2015 1 квартал (январь, февраль, март)испр.xls
2015 2 квартал (апрель, май, июнь)испр.xls
2015 3 квартал ( июль, август, сентябрь)испр.xls
2015 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2016 1 квартал (январь,февраль,март)испр.xls
2016 2 квартал (апрель, май, июнь)испр.xls
2016 3 квартал (июль, август, сентябрь)испр.xls
2016 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2квартал2017.xls
2квартал2017_2.xlsx
3 к

In [5]:
# список всех таблиц из всех файлов
sheetname_list = []
for name, dict_ in df_dicts:    
    tmp = dict_.keys()
    tmp = list(tmp)
    tmp = tmp[2:-3]
    sheetname_list.append(tmp)
len(sheetname_list)

27

In [6]:
# dict_sheet_count - словарь всех вкладок(листов) из таблиц без дубликатов и их частоты
# uniq_sheetname - множество всех вкладок(листов) из таблиц без дубликатов (не используется)

dict_sheet_count = {}
uniq_sheetname = set()
for st in sheetname_list:
    for ls in st:
        uniq_sheetname.add(ls)


uniq_sheetname = list(uniq_sheetname)
uniq_sheetname.sort()
print(len(uniq_sheetname))

# добавление частот в словарь
for st in sheetname_list:
    for ls in st:
        counts = dict_sheet_count.get(ls,0)
        dict_sheet_count[ls] = counts + 1
        
        
# df - для вывода словаря в excel            
df = pd.DataFrame.from_dict(data = dict_sheet_count, orient='index')
df.to_excel('dict_sheet_count.xlsx')

91


In [7]:
# переименование вкладок с одним смыслом в те, которые чаще появлялись

def new_sheetname(ordDict, oldName, newName, fl=False):
    tmp = OrderedDict((newName if k == oldName else k, v) for k, v in ordDict.items())
    return tmp

temp = []
for name, df_l in df_dicts:
    df_l = new_sheetname(df_l, u'Доля вязк. составл.(-20) Шарпи', u'Доля вязк. составл.КСV(-20).')
    df_l = new_sheetname(df_l, u'Доля вязк. составл. КСV(-50).', u'Доля вязк. составл.КСV(-50).')
    df_l = new_sheetname(df_l, u'Доля вязк. составл.(-50) Шарпи',u'Доля вязк. составл.КСV(-50).')
    df_l = new_sheetname(df_l, u'Доля вязк. составл. КСV(-60)',u'Доля вязк. составл.(-60)Шарпи.')
    df_l = new_sheetname(df_l, u'Доля вязк. составл.КСV(-60)',u'Доля вязк. составл.(-60)Шарпи.')
    df_l = new_sheetname(df_l, u'Доля вязк. составл. на КV(0).',u'Доля вязк. составл.КV(0).')
    df_l = new_sheetname(df_l, u'Доля вязк. составл.КСU(-60)',u'Доля вязк. составл.(-60)Менаже.')
    df_l = new_sheetname(df_l, u'Доля вязк. составл.КV(-20).',u'Доля вязк. составл.КСV(-20).')
    df_l = new_sheetname(df_l, u'Работа удара КV0 (ДЖ).',u'Работа удара КV(0) (ДЖ).')
    df_l = new_sheetname(df_l, u'Твердость по Бринеллю НВ',u'Твердость по Бринеллю.')
    df_l = new_sheetname(df_l, u'Твердость по Виккерсу HV.',u'Твердость по Виккерсу.')
    df_l = new_sheetname(df_l, u'Твердость по Виккерсу НV',u'Твердость по Виккерсу.')
    df_l = new_sheetname(df_l, u'Твердость по Роквеллу  HRB.',u'Твердость по Роквеллу HRB.')
    df_l = new_sheetname(df_l, u'Твердость по Роквеллу НRС API.',u'Твердость по Роквеллу API HRC.')
    temp.append([name, df_l])
    
df_dicts = temp     

In [8]:
# среднее значение для вкладок, где несколько испытаний на 1 партию
def get_mean(df_sheet, ls_col):
    if '№ патрубка' in df_sheet:
        df_sheet = df_sheet.drop(['№ патрубка'], axis=1)
    df_sheet = df_sheet.groupby(['№ плавки', '№ партии']).first()
    for col in ls_col:
        if col in df_sheet:
            try:
                df_sheet[col] = df_sheet.groupby(['№ плавки', '№ партии'])[col].transform(lambda x: x.mean())
            except:
                pass
    df_sheet = df_sheet.reset_index()
    return df_sheet

In [9]:
# среднее значение для вкладок, где несколько испытаний на 1 партию, но берем последнюю строчку(наименьшее знач)
def get_mean_last(df_sheet, ls_col):
    if '№ патрубка' in df_sheet:
        df_sheet = df_sheet.drop(['№ патрубка'], axis=1)
    df_sheet = df_sheet.groupby(['№ плавки', '№ партии']).last()
    for col in ls_col:
        if col in df_sheet:
            try:
                df_sheet[col] = df_sheet.groupby(['№ плавки', '№ партии'])[col].transform(lambda x: x.mean())
            except:
                pass
    df_sheet = df_sheet.reset_index()
    return df_sheet

In [10]:
# среднее значение для вкладок растяжений
def get_mean_stretch(df_sheet, ls_col):
    if '№ патрубка' in df_sheet:
        df_sheet = df_sheet.drop([u'№ патрубка'], axis=1)
    if 'Тип предела текучести (1186)' in df_sheet:
        df_sheet = df_sheet.groupby(['Тип предела текучести (1186)', '№ партии']).first()
        for col in ls_col:
            if col in df_sheet:
                df_sheet[col] = df_sheet.groupby(['Тип предела текучести (1186)', '№ партии'])[col].transform(lambda x: x.mean())
        df_sheet = df_sheet.reset_index()
    return df_sheet

In [11]:
# найти файл по вкладке и вывести размер вкладки
def findfile(sheetname):
    row = 0
    col = 0
    for name, df_l in df_dicts:
        try:
            col = df_l[sheetname].shape[1]
            df_l[sheetname]
            print(name)
        except:
            continue
        try:
            row += df_l[sheetname].shape[0] 
            if (abs(col - col)<1):
                   continue
            else:
                col += abs(df_l[sheetname].shape[1] - col)
        except:
            continue
    print('rows = %d, col = %d' %(row, col))


In [12]:
# перевод н, дж в кг
ls_sheets_convert = [
    u'Растяжение, н мм2 , Ksi.', 
    u'Растяжение, н мм2.',
    u'Уд.вязк.(-60) Шарпи(ДЖ).',
    u'Уд.вязк.(-40) Менаже(ДЖ).'
    ]

ls_columns_convert = [
    u'Работа удара, Дж. (1173)',
    u'Ударная вязкость, Дж/см2 (1174)',
    u'Работа удара, Дж. (1123)',
    u'Ударная вязкость, Дж/см2 (1125)'
    ]

def convert(ls_sheets, ls_columns, df):
    for name, df_l in df:
        for sheet in ls_sheets:
            for col in ls_columns:
                if sheet in df_l:
                    if col in df_l[sheet]:
                        df_l[sheet][col] = df_l[sheet][col]/9.80665
                                                
    return df

ls_mb_convert = [
    'Предел текучести (1141)',
    'Предел текучести (1027)',
    'Предел текучести (1027)',
    'Предел текучести, Н/мм2 или МПа',
    'Предел текучести , кгс/мм2',
    'Предел текучести, кгс/мм2',
    
    
    'Врем. сопротивление,  Н/мм2 или МПа',
    'Врем. сопротивление (1140)',
    'Врем. сопротивление (1042)',
    'Врем. сопротивление , кгс/мм2',
    'Врем. сопротивление, кгс/мм2'
    
]

def convert_mb(ls_sheets, ls_columns, df):
    for name, df_l in df:
        for sheet in ls_sheets:
            for col in ls_columns:
                if sheet in df_l:
                    if col in df_l[sheet]:
                        df_l[sheet][col] =  df_l[sheet][col].apply(lambda x: x/9.80665
                                                                   if (x > 200)
                                                                   else x) 
    return df

In [13]:
# конвертация
df_dicts = convert(ls_sheets_convert, ls_columns_convert, df_dicts)
df_dicts = convert_mb(ls_sheets_convert, ls_mb_convert, df_dicts)

In [14]:
# очистка
def rep(x):
    x = str(x)
    x = x.replace('..', '.')
    x = x.replace(',', '.')
    x = x.replace('. ', '.')
    x = x.replace('>', '')
    x = x.replace('.0.5', '0.5')
    x = x.replace('.9.05', '9.05')
    return x

def spl(x):
    fl = True
    x = str(x)
    tmpy = []
    y =  x.split(';')
    for i in y:
        try:
            tmpy.append(float(i))
        except:
            tmpy.append(i)
            fl = False
    if(fl):
        y = tmpy   
        x = np.mean(y)
    return x

In [15]:
# список вкладок важных испытаний
sheet_important = [
    u'Величина зерна.',
    u'Доля вязк. составл.(0).',
    u'Доля вязк. составл.(-20).',
    u'Доля вязк. составл.(-50).',
    u'Доля вязк. составл.(-60)Шарпи.',
    u'Макроструктура (трубы).',
    u'Микроструктура.',
    u'Неметал. включения.',
    u'Прокаливаемость HRC.',
    u'Прокаливаемость для кл.N80 тип.',
    u'Растяжение, кгс мм2.',
    u'Растяжение, н мм2 , Ksi.',
    u'Растяжение, н мм2.',
    u'Сплющивание.',
    u'Твердость по Бринеллю.',
    u'Твердость по Роквеллу API.',
    u'Твердость по Роквеллу HRB.',
    u'Твердость по Роквеллу HRС.',
    u'Уд.вязк(-20) Шарпи.(КГС М С).',
    u'Уд.вязк.(-40) Менаже(ДЖ).',
    u'Уд.вязк.(-40) Менаже(КГС М С).',
    u'Уд.вязк.(-60) Шарпи(ДЖ).',
    u'Уд.вязк.(-60) Шарпи(КГС М С).'
    ]

# список столбцов из важных вкладок, которые необходимо усреднить
col_for_mean = [
    u'Величина зерна (4006)',
    u'% площади сдвига (вязк.сост.) (1138)',
    u'% площади сдвига (вязк.сост.) (1208)',
    u'% площади сдвига (вязк.сост.) (1210)',
    u'% площади сдвига (вязк.сост.) (1212)',
    u'max Единицы твердости (1071)',
    u'Врем. сопротивление (1042)',
    u'Врем. сопротивление (1140)',
    u'Врем. сопротивление, Ksi (1146)',
    u'Диам. отпечатка (8022)',
    u'Диаметр образца (1225)',
    u'Единицы твердости (1032)',
    u'Единицы твердости (1060)',
    u'Единицы твердости (1071)', 
    u'Макроструктура (4030)', # Значения "Уд." и "Дефекты отсутствуют"
    u'Нитриды (2010)',
    u'ОС (оксиды строчечные) (2005)',
    u'ОТ (оксиды точечные) (2004)',
    u'Относительное сужение (1016)',
    u'Относительное удлинение (1099)',
    u'Отношение (1118)',
    u'Полосчатость (4036)',
    u'Предел  текучести, Ksi (1145)',
    u'Предел текучести (1027)',
    u'Предел текучести (1141)',
    u'Работа удара (9051)',
    u'Работа удара, Дж. (1123)',
    u'Работа удара, Дж. (1173)',
    u'С (сульфиды) (2008)',
    u'Силикаты недеформированные (2009)',
    u'Силикаты пластичные (2007)',
    u'Силикаты хрупкие (2006)',
    u'Сплющивание (1119)', # Значение "Трещин нет"
    u'Толщина (1226)',
    u'Ударная вязкость (1073)',
    u'Ударная вязкость (1097)',
    u'Ударная вязкость (1100)',
    u'Ударная вязкость, Дж/см2 (1125)',
    u'Ударная вязкость, Дж/см2 (1174)',
    u'Число твердости на внутренней поверхности (1149)',
    'Число твердости на середине поверхности (1148)',
    u'Число твердости у наружной поверхности (1147)',
    u'Предел текучести, кгс/мм2',
    u'Врем. сопротивление, кгс/мм2',
    u'Предел текучести, Н/мм2 или МПа',
    u'Врем. сопротивление,  Н/мм2 или МПа',
    'Тип предела текучести, Н/мм2 или МПа'
    ]

In [16]:
# очистка
tmpdict = []
for name, df_l in df_dicts:
    for sheet in sheet_important:
        for col in col_for_mean:
            if sheet in df_l:
                if col in df_l[sheet]:
                    df_l[sheet][col] = df_l[sheet][col].map(rep)
                    df_l[sheet][col] = df_l[sheet][col].map(spl)
    tmpdict.append([name, df_l])  
df_dicts = tmpdict

In [17]:
# усреднение
tmp = []
for name, df_l in df_dicts:
    print(name)
    for ls in sheet_important:
        if ls in df_l:
            df_l[ls] = get_mean_stretch(df_l[ls], col_for_mean)
            df_l[ls] = get_mean(df_l[ls], col_for_mean)
    tmp.append([name, df_l])
            
df_dicts = tmp

1квартал2017.xls
2012 1 квартал (январь, февраль, март)испр.xls
2012 2 квартал (апрель, май, июнь)испр..xls
2012 3 квартал (июль, август, сентябрь)испр..xls
2012 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2013 1 квартал (январь, февраль, март)испр.xls
2013 2 квартал (апрель, май, июнь)испр..xls
2013 3 квартал (июль, август, сентябрь)испр..xls
2013 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2014 1 квартал (январь, февраль, март)испр.xls
2014 2 квартал (апрель, май, июнь)испр.xls
2014 3 квартал (июль, август, сентябрь)испр.xls
2014 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2015 1 квартал (январь, февраль, март)испр.xls
2015 2 квартал (апрель, май, июнь)испр.xls
2015 3 квартал ( июль, август, сентябрь)испр.xls
2015 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2016 1 квартал (январь,февраль,март)испр.xls
2016 2 квартал (апрель, май, июнь)испр.xls
2016 3 квартал (июль, август, сентябрь)испр.xls
2016 4 квартал (октябрь, ноябрь, декабрь)испр.xls
2квартал2017.xls
2квартал2017_2.xlsx
3 к

In [18]:
# при усреднении Mr.Py ставит запятые в дробях, так что опять меняем зпт на тчк
# бесполезный шаг, ведь дальше все равно будет куча запятых...
tmpdict = []
for name, df_l in df_dicts:
    for sheet in sheet_important:
        if sheet in df_l:
            for col in df_l[sheet].columns:
                df_l[sheet][col] = df_l[sheet][col].apply(lambda x: str(x).replace(',', '.'))
    tmpdict.append([name, df_l])  
df_dicts = tmpdict

In [19]:
# удаление пустого столбца
for name, df_l in df_dicts:
    for sheet in sheet_important:
        if sheet in df_l:
            if 'Unnamed: 10' in  df_l[sheet].columns:
                df_l[sheet] = df_l[sheet].drop(['Unnamed: 10'], axis=1)


# удаление нулевых столбцов
for name, df_l in df_dicts:
    for sheet in sheet_important:
        if sheet in df_l:
            for col in df_l[sheet].columns:
                if df_l[sheet][col].max() == '0':
                    df_l[sheet] = df_l[sheet].drop([col], axis = 1)

In [24]:
# объединение таблиц с испытаниями
def union_mode_table_valid(union_list, df_dicts, file_name):
    if union_list[0] in sheetname_list_valid:
        table_union = df_dicts[1][union_list[0]]
    else:
        table_union = []
        print('0')
        return table_union
    for series_name in union_list:
        if series_name in sheetname_list_valid:
            table_union = pd.merge(table_union, df_dicts[1][series_name], how = 'outer')
    table_union[u'№ плавки'] = table_union[u'№ плавки'].astype(str)
    print(table_union.shape, file_name)
    return table_union

In [25]:
# объединение таблиц с испытаниями
def union_mode_table(union_list, df_dicts, file_name):
    table_union = df_dicts[0][1][union_list[0]]
    for name, df_l in df_dicts:
        for series_name in union_list:
            if series_name in df_l:
                table_union = pd.merge(table_union, df_l[series_name], how = 'outer')
    table_union[u'№ плавки'] = table_union[u'№ плавки'].astype(str)
    print(table_union.shape, file_name)
    return table_union

In [26]:
# сохранение объединенных таблиц
def save_union(table_union, file_name):
    table_union.to_excel('exp/prepared/'+file_name+'.xlsx')

In [27]:
# таблица с растяжениями
stretching_union_list = [
    'Растяжение, кгс мм2.',
    'Растяжение, н мм2.',
    'Растяжение, н мм2 , Ksi.',
]

# таблица с Уд.вязк.(-60) Шарпи(КГС М С)
resiliency_60_union_list = [
    u'Уд.вязк.(-60) Шарпи(КГС М С).',
    u'Уд.вязк.(-60) Шарпи(ДЖ).' 
]

# таблица с  Уд.вязк.(-40) Менаже(КГС М С)
resiliency_40_union_list =[
    u'Уд.вязк.(-40) Менаже(КГС М С).',
    u'Уд.вязк.(-40) Менаже(ДЖ).'
]


# таблица с Уд.вязк(-20) Шарпи.(КГС М С).
resiliency_20_union_list = [
    u'Уд.вязк(-20) Шарпи.(КГС М С).'   
]

# таблица с прокаливаемостью
hardenability_union_list = [
    u'Прокаливаемость HRC.',
    u'Прокаливаемость для кл.N80 тип.'
]


# Величина зерна
grain_size_union_list = [
    u'Величина зерна.'
]


# таблица с твердостью по Бриннелю
hardness_hb_union_list = [
    u'Твердость по Бринеллю.',
]

# таблица с долей вязкой составляющей 0
proportion_viscous_0_union_list = [
    u'Доля вязк. составл.(0).'
]

# таблица с Доля вязк. составл.(-20)
proportion_viscous_20_union_list = [
    u'Доля вязк. составл.(-20).'
]


# таблица с Доля вязк. составл.(-50).
proportion_viscous_50_union_list = [
    u'Доля вязк. составл.(-50).'
]


# таблица с Доля вязк. составл.(-60)Шарпи.
proportion_viscous_60_union_list = [
    u'Доля вязк. составл.(-60)Шарпи.'
]

# таблица с u'Макроструктура (трубы).'
macrostructure_union_list = [
    u'Макроструктура (трубы).'
]
# таблица с микроструктурой
microstructure_union_list = [
    u'Микроструктура.'
]

# таблица с  u'Неметал. включения.',
nonmetall_union_list = [
     u'Неметал. включения.'
]

# таблица с u'Сплющивание.'
flattening_union_list = [
    u'Сплющивание.'
]

# таблица с u'Твердость по Роквеллу API.'
hardness_api_union_list = [
    u'Твердость по Роквеллу API.'
]
# таблица u'Твердость по Роквеллу HRС.'
hardness_hrc_union_list = [
    u'Твердость по Роквеллу HRС.'
]
# таблица с u'Твердость по Роквеллу HRB.',
hardness_hrb_union_list = [
    u'Твердость по Роквеллу HRB.'
]


table_hardness_hrb = union_mode_table(hardness_hrb_union_list, df_dicts[:25], 'table_hardness_hrb')
table_hardness_hrc = union_mode_table(hardness_hrc_union_list, df_dicts[:25], 'table_hardness_hrc')
table_hardness_api = union_mode_table(hardness_api_union_list, df_dicts[:25], 'table_hardness_api')
table_flattening = union_mode_table(flattening_union_list, df_dicts[:25], 'table_flattening')
table_nonmetall = union_mode_table(nonmetall_union_list, df_dicts[:25], 'table_nonmetall')
table_macrostructure = union_mode_table(macrostructure_union_list, df_dicts[:25], 'table_macrostructure')
table_microstructure = union_mode_table(microstructure_union_list, df_dicts[:25], 'table_microstructure')
table_proportion_viscous_60 = union_mode_table(proportion_viscous_60_union_list, df_dicts[:25], 'table_proportion_viscous_60')
table_proportion_viscous_50 = union_mode_table(proportion_viscous_50_union_list, df_dicts[:25], 'table_proportion_viscous_50')
table_proportion_viscous_20 = union_mode_table(proportion_viscous_20_union_list, df_dicts[:25], 'table_proportion_viscous_20')
table_proportion_viscous_0 = union_mode_table(proportion_viscous_0_union_list, df_dicts[:25], 'table_proportion_viscous_0')
table_hardness_hb = union_mode_table(hardness_hb_union_list, df_dicts[:25], 'table_hardness_hb')
table_grain_size = union_mode_table(grain_size_union_list, df_dicts[:25], 'table_grain_size')    
table_hardenability = union_mode_table(hardenability_union_list, df_dicts[:25], 'table_hardenability')
table_resiliency_20 = union_mode_table(resiliency_20_union_list, df_dicts[:25], 'table_resiliency_20')
table_resiliency_40 = union_mode_table(resiliency_40_union_list, df_dicts[:25], 'table_resiliency_40')
table_stretching = union_mode_table(stretching_union_list, df_dicts[:25], 'table_stretching')
table_resiliency_60 = union_mode_table(resiliency_60_union_list, df_dicts[:25], 'table_resiliency_60')

(2543, 14) table_hardness_hrb
(894, 10) table_hardness_hrc
(3090, 12) table_hardness_api
(10248, 10) table_flattening
(4776, 16) table_nonmetall
(343, 10) table_macrostructure
(4976, 12) table_microstructure
(419, 10) table_proportion_viscous_60
(4720, 11) table_proportion_viscous_50
(2783, 10) table_proportion_viscous_20
(158, 10) table_proportion_viscous_0
(709, 12) table_hardness_hb
(5249, 10) table_grain_size
(590, 12) table_hardenability
(175, 12) table_resiliency_20
(211, 13) table_resiliency_40
(31132, 34) table_stretching
(700, 13) table_resiliency_60


In [28]:
table_hardness_hrb_valid = union_mode_table_valid(hardness_hrb_union_list, df_dicts[26], 'table_hardness_hrb_valid')
table_hardness_hrc_valid = union_mode_table_valid(hardness_hrc_union_list, df_dicts[26], 'table_hardness_hrc_valid')
table_hardness_api_valid = union_mode_table_valid(hardness_api_union_list, df_dicts[26], 'table_hardness_api_valid')
table_flattening_valid = union_mode_table_valid(flattening_union_list, df_dicts[26], 'table_flattening_valid')
table_nonmetall_valid = union_mode_table_valid(nonmetall_union_list, df_dicts[26], 'table_nonmetall_valid')
table_macrostructure_valid = union_mode_table_valid(macrostructure_union_list, df_dicts[26], 'table_macrostructure_valid')
table_microstructure_valid = union_mode_table_valid(microstructure_union_list, df_dicts[26], 'table_microstructure_valid')
table_proportion_viscous_60_valid = union_mode_table_valid(proportion_viscous_60_union_list, df_dicts[26], 'table_proportion_viscous_60_valid')
table_proportion_viscous_50_valid = union_mode_table_valid(proportion_viscous_50_union_list, df_dicts[26], 'table_proportion_viscous_50_valid')
table_proportion_viscous_20_valid = union_mode_table_valid(proportion_viscous_20_union_list, df_dicts[26], 'table_proportion_viscous_20_valid')
table_proportion_viscous_0_valid = union_mode_table_valid(proportion_viscous_0_union_list, df_dicts[26], 'table_proportion_viscous_0_valid')
table_hardness_hb_valid = union_mode_table_valid(hardness_hb_union_list, df_dicts[26], 'table_hardness_hb_valid')
table_grain_size_valid = union_mode_table_valid(grain_size_union_list, df_dicts[26], 'table_grain_size_valid')    
table_hardenability_valid = union_mode_table_valid(hardenability_union_list, df_dicts[26], 'table_hardenability_valid')
table_resiliency_20_valid = union_mode_table_valid(resiliency_20_union_list, df_dicts[26], 'table_resiliency_20_valid')
table_resiliency_40_valid = union_mode_table_valid(resiliency_40_union_list, df_dicts[26], 'table_resiliency_40_valid')
table_stretching_valid = union_mode_table_valid(stretching_union_list, df_dicts[26], 'table_stretching_valid')
table_resiliency_60_valid = union_mode_table_valid(resiliency_60_union_list, df_dicts[26], 'table_resiliency_60_valid')

(1, 11) table_hardness_hrb_valid
(3, 10) table_hardness_hrc_valid
(9, 12) table_hardness_api_valid
(207, 10) table_flattening_valid
(34, 16) table_nonmetall_valid
0
(25, 10) table_microstructure_valid
(9, 10) table_proportion_viscous_60_valid
(16, 11) table_proportion_viscous_50_valid
(127, 10) table_proportion_viscous_20_valid
(6, 10) table_proportion_viscous_0_valid
0
(26, 10) table_grain_size_valid
(1, 12) table_hardenability_valid
(23, 12) table_resiliency_20_valid
(21, 11) table_resiliency_40_valid
(629, 22) table_stretching_valid
0


In [29]:
# список всех объединенных таблиц
ls_table_ = [
    table_hardness_hrb,
    table_hardness_hrc,
    table_hardness_api,
    table_flattening,
    table_nonmetall,
    table_macrostructure,
    table_microstructure,
    table_proportion_viscous_60,
    table_proportion_viscous_50,
    table_proportion_viscous_20,
    table_proportion_viscous_0,
    table_hardness_hb,
    table_grain_size,
    table_hardenability,
    table_resiliency_20,
    table_resiliency_40,
    table_stretching,
    table_resiliency_60
]

# список имен объединенных таблиц
ls_filename = [
    'table_hardness_hrb',
    'table_hardness_hrc',
    'table_hardness_api',
    'table_flattening',
    'table_nonmetall',
    'table_macrostructure',
    'table_microstructure',
    'table_proportion_viscous_60',
    'table_proportion_viscous_50',
    'table_proportion_viscous_20',
    'table_proportion_viscous_0',
    'table_hardness_hb',
    'table_grain_size',
    'table_hardenability',
    'table_resiliency_20',
    'table_resiliency_40',
    'table_stretching',
    'table_resiliency_60'
]

ls_table_valid = [
    table_hardness_hrb_valid,
    table_hardness_hrc_valid,
    table_hardness_api_valid,
    table_flattening_valid,
    table_nonmetall_valid,
    table_microstructure_valid,
    table_proportion_viscous_60_valid,
    table_proportion_viscous_50_valid,
    table_proportion_viscous_20_valid,
    table_proportion_viscous_0_valid,
    table_grain_size_valid,
    table_hardenability_valid,
    table_resiliency_20_valid,
    table_resiliency_40_valid,
    table_stretching_valid
]

# список имен объединенных таблиц
ls_filename_valid = [
    'table_hardness_hrb_valid',
    'table_hardness_hrc_valid',
    'table_hardness_api_valid',
    'table_flattening_valid',
    'table_nonmetall_valid',
    'table_microstructure_valid',
    'table_proportion_viscous_60_valid',
    'table_proportion_viscous_50_valid',
    'table_proportion_viscous_20_valid',
    'table_proportion_viscous_0_valid',
    'table_grain_size_valid',
    'table_hardenability_valid',
    'table_resiliency_20_valid',
    'table_resiliency_40_valid',
    'table_stretching_valid'
]

In [30]:
# # сохранить объединенные таблицы с дубликатами
# for i in range(0, len(ls_table_testing)):
#     save_union(ls_table_testing[i], 'with duplicates/'+ls_filename[i])
#     print(ls_filename[i])

In [31]:
# ради одной строчки в table_proportion_viscous_60 (берем наименьшее значение 17.7, а не 100)
ls_table_[7] = get_mean_last(ls_table_[7], col_for_mean)

In [32]:
# приведение всех партии к нижнему регистру(для присоединения с режимами и химией)
def low(ls):
    tmp = []
    for df_l in ls:
        if '№ партии' in df_l:
            df_l['№ партии'] = df_l['№ партии'].astype(str).str.lower()
            tmp.append(df_l)    
        print(df_l.shape[0])
    return tmp

In [33]:
ls_table_ = low(ls_table_)
print('\ntables for validation:')
ls_table_valid = low(ls_table_valid)

2543
894
3090
10248
4776
343
4976
418
4720
2783
158
709
5249
590
175
211
31132
700

tables for validation:
1
3
9
207
34
25
9
16
127
6
26
1
23
21
629


In [34]:
# усреднение объединенных таблиц
def mean_exp(ls_table):
    tmp = []
    for df_l in ls_table:
        df_l = get_mean_stretch(df_l, col_for_mean)
        df_l = get_mean(df_l, col_for_mean)
        tmp.append(df_l)    
        print(df_l.shape[0])
    return tmp

In [35]:
ls_table_ = mean_exp(ls_table_)
print('\ntables for validation:')
ls_table_valid = mean_exp(ls_table_valid)

2541
870
3089
10200
4772
343
4945
418
4710
2780
158
709
5218
582
175
211
28908
697

tables for validation:
1
3
9
207
34
25
9
16
127
6
26
1
23
21
576


In [36]:
col_fluidity = [
    'Врем. сопротивление (1140)',
    'Врем. сопротивление (1042)',
    'Врем. сопротивление,  Н/мм2 или МПа',
    'Врем. сопротивление , кгс/мм2',
    'Врем. сопротивление, кгс/мм2',
    'Предел текучести (1141)',
    'Предел текучести (1027)',
    'Предел текучести, Н/мм2 или МПа',
    'Предел текучести , кгс/мм2',
    'Предел текучести, кгс/мм2'
]

for col in col_fluidity:
    if col in table_stretching_valid:
        table_stretching_valid[col] = table_stretching_valid[col].apply(lambda x: str(x))
    else:
        print(col)
    table_stretching[col] = table_stretching[col].apply(lambda x: str(x))

Врем. сопротивление,  Н/мм2 или МПа
Врем. сопротивление , кгс/мм2
Врем. сопротивление, кгс/мм2
Предел текучести, Н/мм2 или МПа
Предел текучести , кгс/мм2
Предел текучести, кгс/мм2


In [37]:
"""На всякий случай оставляю, если я где-нибудь накосячила..... """

#             elif ('Врем. сопротивление (1142)' in table):
#                 if (table['Врем. сопротивление (1042)'][x] != 'nan'):
#                     tmp.append(table['Врем. сопротивление (1042)'][x])
#                 elif ('Врем. сопротивление,  Н/мм2 или МПа' in table):
#                     if (table['Врем. сопротивление,  Н/мм2 или МПа'][x] != 'nan'):
#                         tmp.append(table['Врем. сопротивление,  Н/мм2 или МПа'][x])
#                     elif ('Врем. сопротивление , кгс/мм2' in table):
#                         if (table['Врем. сопротивление , кгс/мм2'][x] != 'nan'):
#                             tmp.append(table['Врем. сопротивление , кгс/мм2'][x])
#                         elif ('Врем. сопротивление, кгс/мм2' in table):
#                             if (table['Врем. сопротивление, кгс/мм2'][x] != 'nan'):
#                                 tmp.append(table['Врем. сопротивление, кгс/мм2'][x])
#                             else:
#                                 tmp.append('')



#     for x in range(0, table.shape[0]):
#         if 'Предел текучести (1141)' in table:
#             if (table['Предел текучести (1141)'][x] != 'nan'):
#                 tmp.append(table['Предел текучести (1141)'][x])
#             else:
#                 if 'Предел текучести (1027)' in table:
#                     if (table['Предел текучести (1027)'][x] != 'nan'):
#                         tmp.append(table['Предел текучести (1027)'][x])
#                     else:
#                         if 'Предел текучести, Н/мм2 или МПа' in table:
#                             if (table['Предел текучести, Н/мм2 или МПа'][x] != 'nan'):
#                                 tmp.append(table['Предел текучести, Н/мм2 или МПа'][x])
#                             else:
#                                 if 'Предел текучести, Н/мм2 или МПа' in table:
#                                     if (table['Предел текучести, Н/мм2 или МПа'][x] != 'nan'):
#                                         tmp.append(table['Предел текучести, Н/мм2 или МПа'][x])
#                                     else:
#                                         if 'Предел текучести, кгс/мм2' in table:
#                                             if (table['Предел текучести, кгс/мм2'][x] != 'nan'):
#                                                 tmp.append(table['Предел текучести, кгс/мм2'][x])
#                                             else:
#                                                 tmp.append('')
 

'На всякий случай оставляю, если я где-нибудь накосячила..... '

In [49]:
# Схлопываем несколько столбцов с текучестью и прочность в один для каждого
list_tr_col_names = [
    'Врем. сопротивление',
    'Врем. сопротивление (1140)',
    'Врем. сопротивление (1042)',
    'Врем. сопротивление,  Н/мм2 или МПа',
    'Врем. сопротивление , кгс/мм2',
    'Врем. сопротивление, кгс/мм2'
]

list_ys_col_names = [
    'Предел текучести',
    'Предел текучести (1141)',
    'Предел текучести (1027)',
    'Предел текучести, Н/мм2 или МПа',
    'Предел текучести , кгс/мм2',
    'Предел текучести, кгс/мм2'
]


def merge_col(table, list_col_names):
    """Схлопывание колонок.

    Keyword arguments:
    table -- датафрейм в котором будем сливать столбцы
    list_col_names -- список названий колонок 
        (первым в списке должно идти имя столбца в который будет записан конечный результат)

    Возвращает преобразованный датафрейм с добавленым столбцом
    """
    tmp = []
    for x in range(0, table.shape[0]):
        fl = True
        for col_name in list_col_names[1:]:
            if fl & (col_name in table):
                if (table[col_name][x] != 'nan'):
                    tmp.append(table[col_name][x])
                    fl = False
        if fl:
            tmp.append('')
            
    print(len(tmp), table.shape )
    table[list_col_names[0]] = tmp
    return table
    
    
def col_merge(table):
    """Сливает столбцы текучести в один, сливает столбцы прочности в один"""
    table = merge_col(table, list_tr_col_names)
    table = merge_col(table, list_ys_col_names)
    return table

In [50]:
table_stretching.shape

(31132, 35)

In [51]:
table_stretching = col_merge(table_stretching)
table_stretching_valid = col_merge(table_stretching_valid)

31132 (31132, 35)
31132 (31132, 35)
629 (629, 23)
629 (629, 23)


In [53]:
table_stretching = table_stretching[table_stretching['Предел текучести'] != '']
table_stretching = table_stretching[table_stretching['Врем. сопротивление'] != '']
table_stretching_valid = table_stretching_valid[table_stretching_valid['Предел текучести'] != '']
table_stretching_valid = table_stretching_valid[table_stretching_valid['Врем. сопротивление'] != '']

In [54]:
for x in table_stretching['Врем. сопротивление']:
    try:
        x = float(x)
    except:
        print(x)
        
for x in table_stretching['Предел текучести']:
    try:
        x = float(x)
    except:
        print(x)
table_stretching['Предел текучести'] = table_stretching['Предел текучести'].apply(lambda x: float(x))
table_stretching['Врем. сопротивление'] = table_stretching['Врем. сопротивление'].apply(lambda x: float(x))
table_stretching_valid['Предел текучести'] = table_stretching_valid['Предел текучести'].apply(lambda x: float(x))
table_stretching_valid['Врем. сопротивление'] = table_stretching_valid['Врем. сопротивление'].apply(lambda x: float(x))

In [55]:
table_stretching['Предел текучести'] = table_stretching['Предел текучести'].apply(lambda x: x/9.80665
                                                                   if (x > 200)
                                                                   else x)
table_stretching['Врем. сопротивление'] = table_stretching['Врем. сопротивление'].apply(lambda x: x/9.80665
                                                                   if (x > 200)
                                                                   else x)
table_stretching_valid['Предел текучести'] = table_stretching_valid['Предел текучести'].apply(lambda x: x/9.80665
                                                                   if (x > 200)
                                                                   else x)
table_stretching_valid['Врем. сопротивление'] = table_stretching_valid['Врем. сопротивление'].apply(lambda x: x/9.80665
                                                                   if (x > 200)
                                                                   else x)

In [66]:
ls_table_ = [
    table_hardness_hrb,
    table_hardness_hrc,
    table_hardness_api,
    table_flattening,
    table_nonmetall,
    table_macrostructure,
    table_microstructure,
    table_proportion_viscous_60,
    table_proportion_viscous_50,
    table_proportion_viscous_20,
    table_proportion_viscous_0,
    table_hardness_hb,
    table_grain_size,
    table_hardenability,
    table_resiliency_20,
    table_resiliency_40,
    table_stretching,
    table_resiliency_60
]

ls_table_valid = [
    table_hardness_hrb_valid,
    table_hardness_hrc_valid,
    table_hardness_api_valid,
    table_flattening_valid,
    table_nonmetall_valid,
    table_microstructure_valid,
    table_proportion_viscous_60_valid,
    table_proportion_viscous_50_valid,
    table_proportion_viscous_20_valid,
    table_proportion_viscous_0_valid,
    table_grain_size_valid,
    table_hardenability_valid,
    table_resiliency_20_valid,
    table_resiliency_40_valid,
    table_stretching_valid
]

In [67]:
# сохранение усредненных объединенных таблиц
for i in range(0, len(ls_table_)):
    save_union(ls_table_[i], ls_filename[i])
    print(ls_filename[i])

table_hardness_hrb
table_hardness_hrc
table_hardness_api
table_flattening
table_nonmetall
table_macrostructure
table_microstructure
table_proportion_viscous_60
table_proportion_viscous_50
table_proportion_viscous_20
table_proportion_viscous_0
table_hardness_hb
table_grain_size
table_hardenability
table_resiliency_20
table_resiliency_40
table_stretching
table_resiliency_60


In [69]:
for i in range(0, len(ls_table_valid)):
    save_union(ls_table_valid[i], ls_filename_valid[i])
    print(ls_filename_valid[i])

table_hardness_hrb_valid
table_hardness_hrc_valid
table_hardness_api_valid
table_flattening_valid
table_nonmetall_valid
table_microstructure_valid
table_proportion_viscous_60_valid
table_proportion_viscous_50_valid
table_proportion_viscous_20_valid
table_proportion_viscous_0_valid
table_grain_size_valid
table_hardenability_valid
table_resiliency_20_valid
table_resiliency_40_valid
table_stretching_valid


In [68]:
print(table_stretching['Предел текучести'].describe())
print(table_stretching['Врем. сопротивление'].describe())

count    31091.000000
mean        65.940136
std         12.723426
min         23.500000
25%         62.508604
50%         67.148313
75%         71.889993
max        139.000000
Name: Предел текучести, dtype: float64
count    31091.000000
mean        76.481911
std         11.189533
min         38.500000
25%         73.419567
50%         76.733645
75%         82.000000
max        130.000000
Name: Врем. сопротивление, dtype: float64


In [60]:
table_stretching.shape

(31091, 36)

In [93]:
# df_sheet = []
# k=-0
unique_gr_pr = set()
for name, df_l in df_dicts:
    for key in df_l:
        df_sheet = df_l[key]
        try:
            for i in df_sheet[u'Гр. прочн.'].values:
                if i==u'К50/1': i=u'К50-1'
                if i==u'К52/1': i=u'К52-1'
                if i==u'L80 Тип 1': i=u'L80 тип 1'
                unique_gr_pr.add(i)
        except:
            continue
unique_gr_pr = [x for x in unique_gr_pr if x==x]
for i in list(unique_gr_pr):
    print( i)
# print k

X42QS
J55
К60-3
L80SS
К52-1
C90 
Т
P110
C 95 
Д
К50-1
K55
Е
Кс
К52
К54-3
N80 тип Q
М
P110 т13Cr
К
Лх
К55-3
Ехл
К48/3
К48
N80 тип 1
nan
L80тип13Cr
L80 тип 1
Л
L415
Р
C90 тип 1
К72
Т95 тип1  
R95 
К42/3
L290Q
